In [1]:
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport
import json

# Event

In [2]:
df = pd.read_json("./data/event.json")
df2 = pd.DataFrame.from_records(df['Items'])
df2.columns

Index(['Id', 'Pdf', 'Ranc', 'Self', 'Type', '_Meta', 'Active', 'Detail',
       'OrgRID', 'PayMet', 'SignOn', 'Source', 'Ticket', 'Topics', 'DateEnd',
       'Gpstype', 'Hashtag', 'LTSTags', 'Mapping', 'ODHTags', 'SmgTags',
       'Altitude', 'GrpEvent', 'Latitude', 'DateBegin', 'Districts',
       'EventDate', 'GpsPoints', 'Longitude', 'OdhActive', 'Shortname',
       'SmgActive', 'TopicRIDs', 'DistrictId', 'EventPrice', 'LastChange',
       'DistrictIds', 'EventPrices', 'FirstImport', 'HasLanguage',
       'LicenseInfo', 'PublishedOn', 'ContactInfos', 'DistanceInfo',
       'EventBenefit', 'EventBooking', 'ImageGallery', 'LocationInfo',
       'EventDatesEnd', 'NextBeginDate', 'EventPublisher', 'OrganizerInfos',
       'EventDatesBegin', 'EventDateCounter', 'ClassificationRID',
       'EventCrossSelling', 'EventDescAdditional', 'EventAdditionalInfos',
       'AltitudeUnitofMeasure', 'EventOperationScheduleOverview'],
      dtype='object')

In [3]:
df3 = df2[['Id', 'Detail', 'PayMet', 'Source', 'DateEnd', 'Latitude', 'DateBegin', 'Longitude', 'Shortname', 'LocationInfo', 'ContactInfos']]
df3.reset_index(inplace=True)
df3

,index,Id,Detail,PayMet,Source,DateEnd,Latitude,DateBegin,Longitude,Shortname,LocationInfo,ContactInfos
0,0,BFEB2DDB0FD54AC9BC040053A5514A92_REDUCED,"{'it': {'Title': 'Venosta: La ""valle dei 1000 ...",None,lts,2022-06-01T00:00:00,46.644273,2022-06-01T00:00:00,11.225259,1000-Stufen-Schlucht im Vinschgau,{'TvInfo': {'Id': '5228229151CA11D18F1400A0242...,"{'it': {'Tax': None, 'Url': '', 'Vat': None, '..."
1,1,982011E3BE704D22A84F7C2EB40CA8BE_REDUCED,{'it': {'Title': '100 anni banda musicale di c...,None,lts,2023-06-03T00:00:00,46.629600,2023-06-02T00:00:00,10.755800,100 Jahre Musikkapelle Kortsch Jubiläumsfest,{'TvInfo': {'Id': '522822E951CA11D18F1400A0242...,"{'it': {'Tax': None, 'Url': 'https://www.musik..."
2,2,429F65276B0144FCB4ADC354A29C1435_REDUCED,{'it': {'Title': '100 anni banda musicale di C...,None,lts,2022-05-28T00:00:00,46.995900,2022-05-28T00:00:00,11.980500,100-Jahr Jubiläum der Musikkapelle Steinhaus,{'TvInfo': {'Id': '522822F751CA11D18F1400A0242...,"{'it': {'Tax': None, 'Url': '', 'Vat': None, '..."
3,3,10CF80CE69EC42349E4F0B89191F5944_REDUCED,{'it': {'Title': '100 anni banda musicale di C...,None,lts,2022-05-29T00:00:00,46.995940,2022-05-29T00:00:00,11.980515,100-Jahr Jubiläum der Musikkapelle Steinhaus,{'TvInfo': {'Id': '522822F751CA11D18F1400A0242...,"{'it': {'Tax': None, 'Url': '', 'Vat': None, '..."
4,4,9D4618A013144230B8CFF19943335180_REDUCED,{'it': {'Title': '10. Miglio delle prugne a Ba...,None,lts,2022-09-03T00:00:00,46.603100,2022-09-03T00:00:00,11.520300,10. Barbianer Zwetschkenmeile,{'TvInfo': {'Id': '522822A151CA11D18F1400A0242...,"{'it': {'Tax': None, 'Url': 'http://www.klause..."
...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,5C4F708C5B5740D6B2A6DB4A5D6919D3_REDUCED,"{'it': {'Title': '41° Marcia Notturna', 'Heade...",None,lts,2016-08-14T00:00:00,46.724500,2016-08-14T00:00:00,12.224300,41. Toblacher Nachtlauf,{'TvInfo': {'Id': '522822A751CA11D18F1400A0242...,"{'it': {'Tax': None, 'Url': 'http://www.dobbia..."
96,96,1FC5A2ECA6A14AF2B259F529D46FE41F_REDUCED,{'it': {'Title': '41°Gran Fondo Dobbiaco - Cor...,None,lts,2018-02-04T00:00:00,46.735900,2018-02-03T00:00:00,12.222500,41. Volkslanglauf Toblach-Cortina: 50 km Class...,{'TvInfo': {'Id': '522822A751CA11D18F1400A0242...,"{'it': {'Tax': None, 'Url': 'http://www.dobbia..."
97,97,6D2C08F628F3453C87C25FCB2625A10F_REDUCED,{'it': {'Title': '43° concerto della banda mus...,None,lts,2022-04-30T00:00:00,46.626700,2022-04-30T00:00:00,10.772900,43. Festkonzert der Musikkapelle Kortsch,{'TvInfo': {'Id': '522822E951CA11D18F1400A0242...,"{'it': {'Tax': None, 'Url': 'https://www.musik..."
98,98,398AAACC7943414E86A866B5ABB8281A_REDUCED,"{'it': {'Title': '44° Carnevale di Laives', 'H...",None,lts,2023-02-12T00:00:00,46.427100,2023-02-12T00:00:00,11.337200,44. Leiferer Fasching,{'TvInfo': {'Id': '522822B351CA11D18F1400A0242...,"{'it': {'Tax': None, 'Url': 'http://www.carnev..."


In [4]:
# Verrà mergiata
detail = pd.json_normalize(df3['Detail'])
detail.rename(columns={"it.Title": "Title"}, inplace=True)
detail = detail[['Title']]
detail.reset_index(inplace=True)
detail

,index,Title
0,0,"Venosta: La ""valle dei 1000 gradini"""
1,1,100 anni banda musicale di corzes
2,2,100 anni banda musicale di Cadipietra - festa
3,3,100 anni banda musicale di Cadipietra - festa
4,4,10. Miglio delle prugne a Barbiano
...,...,...
95,95,41° Marcia Notturna
96,96,41°Gran Fondo Dobbiaco - Cortina: 50 km Classi...
97,97,43° concerto della banda musicale di Corzes
98,98,44° Carnevale di Laives


In [5]:
munici = pd.json_normalize(df3['LocationInfo'])
munici.rename(columns={"MunicipalityInfo.Name.it": "City"}, inplace=True)
munici = munici[['City']]
munici.reset_index(inplace=True)
munici
munici

,index,City
0,0,Avelengo
1,1,Silandro
2,2,Valle Aurina
3,3,Valle Aurina
4,4,Barbiano
...,...,...
95,95,Dobbiaco
96,96,Dobbiaco
97,97,Silandro
98,98,Laives


In [6]:
zip = pd.json_normalize(df3['ContactInfos'])
zip.rename(columns={"it.ZipCode": "Zip"}, inplace=True)
zip = zip[['Zip']]
zip.reset_index(inplace=True)
zip

,index,Zip
0,0,39010
1,1,39028
2,2,39030
3,3,39030
4,4,39040
...,...,...
95,95,39034
96,96,39034
97,97,39028
98,98,39055


In [7]:
city = pd.merge(munici, zip)
city

,index,City,Zip
0,0,Avelengo,39010
1,1,Silandro,39028
2,2,Valle Aurina,39030
3,3,Valle Aurina,39030
4,4,Barbiano,39040
...,...,...,...
95,95,Dobbiaco,39034
96,96,Dobbiaco,39034
97,97,Silandro,39028
98,98,Laives,39055


In [8]:
detail = pd.merge(detail, city)
detail

,index,Title,City,Zip
0,0,"Venosta: La ""valle dei 1000 gradini""",Avelengo,39010
1,1,100 anni banda musicale di corzes,Silandro,39028
2,2,100 anni banda musicale di Cadipietra - festa,Valle Aurina,39030
3,3,100 anni banda musicale di Cadipietra - festa,Valle Aurina,39030
4,4,10. Miglio delle prugne a Barbiano,Barbiano,39040
...,...,...,...,...
95,95,41° Marcia Notturna,Dobbiaco,39034
96,96,41°Gran Fondo Dobbiaco - Cortina: 50 km Classi...,Dobbiaco,39034
97,97,43° concerto della banda musicale di Corzes,Silandro,39028
98,98,44° Carnevale di Laives,Laives,39055


In [9]:
final = pd.merge(df3, detail)
event = final[['Id', 'Source', 'DateBegin', 'DateEnd', 'Latitude', 'Longitude', 'Shortname', 'Title', 'City', 'Zip']]
event

,Id,Source,DateBegin,DateEnd,Latitude,Longitude,Shortname,Title,City,Zip
0,BFEB2DDB0FD54AC9BC040053A5514A92_REDUCED,lts,2022-06-01T00:00:00,2022-06-01T00:00:00,46.644273,11.225259,1000-Stufen-Schlucht im Vinschgau,"Venosta: La ""valle dei 1000 gradini""",Avelengo,39010
1,982011E3BE704D22A84F7C2EB40CA8BE_REDUCED,lts,2023-06-02T00:00:00,2023-06-03T00:00:00,46.629600,10.755800,100 Jahre Musikkapelle Kortsch Jubiläumsfest,100 anni banda musicale di corzes,Silandro,39028
2,429F65276B0144FCB4ADC354A29C1435_REDUCED,lts,2022-05-28T00:00:00,2022-05-28T00:00:00,46.995900,11.980500,100-Jahr Jubiläum der Musikkapelle Steinhaus,100 anni banda musicale di Cadipietra - festa,Valle Aurina,39030
3,10CF80CE69EC42349E4F0B89191F5944_REDUCED,lts,2022-05-29T00:00:00,2022-05-29T00:00:00,46.995940,11.980515,100-Jahr Jubiläum der Musikkapelle Steinhaus,100 anni banda musicale di Cadipietra - festa,Valle Aurina,39030
4,9D4618A013144230B8CFF19943335180_REDUCED,lts,2022-09-03T00:00:00,2022-09-03T00:00:00,46.603100,11.520300,10. Barbianer Zwetschkenmeile,10. Miglio delle prugne a Barbiano,Barbiano,39040
...,...,...,...,...,...,...,...,...,...,...
95,5C4F708C5B5740D6B2A6DB4A5D6919D3_REDUCED,lts,2016-08-14T00:00:00,2016-08-14T00:00:00,46.724500,12.224300,41. Toblacher Nachtlauf,41° Marcia Notturna,Dobbiaco,39034
96,1FC5A2ECA6A14AF2B259F529D46FE41F_REDUCED,lts,2018-02-03T00:00:00,2018-02-04T00:00:00,46.735900,12.222500,41. Volkslanglauf Toblach-Cortina: 50 km Class...,41°Gran Fondo Dobbiaco - Cortina: 50 km Classi...,Dobbiaco,39034
97,6D2C08F628F3453C87C25FCB2625A10F_REDUCED,lts,2022-04-30T00:00:00,2022-04-30T00:00:00,46.626700,10.772900,43. Festkonzert der Musikkapelle Kortsch,43° concerto della banda musicale di Corzes,Silandro,39028
98,398AAACC7943414E86A866B5ABB8281A_REDUCED,lts,2023-02-12T00:00:00,2023-02-12T00:00:00,46.427100,11.337200,44. Leiferer Fasching,44° Carnevale di Laives,Laives,39055


In [10]:
event.to_csv("./data/raw_csv/event.csv", index=False)

# Accomodation

In [11]:
df = pd.read_json("./data/accomodation.json")
df2 = pd.DataFrame.from_records(df['Items'])
df2.columns

Index(['Id', 'Self', 'HgvId', '_Meta', 'Active', 'Source', 'Gpstype',
       'HasRoom', 'Mapping', 'ODHTags', 'SmgTags', 'AccoType', 'Altitude',
       'BadgeIds', 'BoardIds', 'Features', 'Latitude', 'ThemeIds', 'GpsPoints',
       'IsCamping', 'Longitude', 'OdhActive', 'Shortname', 'SmgActive',
       'AccoBadges', 'AccoBoards', 'AccoDetail', 'AccoThemes', 'AccoTypeId',
       'DistrictId', 'IsBookable', 'LastChange', 'FirstImport', 'HasLanguage',
       'LicenseInfo', 'PublishedOn', 'AccoCategory', 'AccoRoomInfo',
       'DistanceInfo', 'GastronomyId', 'HasApartment', 'ImageGallery',
       'IsGastronomy', 'LocationInfo', 'MainLanguage', 'AccoCategoryId',
       'IndependentData', 'IsAccommodation', 'TourismVereinId',
       'MssResponseShort', 'MarketingGroupIds', 'AccoBookingChannel',
       'SpecialFeaturesIds', 'AccoSpecialFeatures', 'AltitudeUnitofMeasure'],
      dtype='object')

In [12]:
df3 = df2[['Id', 'Source', 'HasRoom', 'AccoTypeId', 'Latitude', 'AccoDetail', 'Longitude', 'Shortname', 'IsBookable']]
df3.reset_index(inplace=True)
df3

,index,Id,Source,HasRoom,AccoTypeId,Latitude,AccoDetail,Longitude,Shortname,IsBookable
0,0,5CEA544EE34639034F07B79D4AEEB603_REDUCED,LTS,False,BedBreakfast,46.477851,"{'de': {'Fax': None, 'Vat': None, 'Zip': '3910...",11.369939,1100 APARTMENT,False
1,1,2657B7CBCB85380B253D2FBE28AF100E_REDUCED,LTS,True,BedBreakfast,46.615101,"{'de': {'Fax': None, 'Vat': None, 'Zip': '3901...",11.143296,1477 Reichhalter,False
2,2,5E4BEFD440200CE4732A7FA30897C402_REDUCED,LTS,False,BedBreakfast,46.902639,"{'de': {'Fax': None, 'Vat': None, 'Zip': '3904...",11.461289,164 apt.,False
3,3,14AF6F5E67CBEAC34F3223D8DE20D08D_REDUCED,LTS,False,BedBreakfast,46.679733,"{'de': {'Fax': None, 'Vat': None, 'Zip': '3901...",11.149205,18,False
4,4,C93CE627A8497E697C657CF53028F9D2_REDUCED,LTS,False,HotelPension,46.660809,"{'de': {'Fax': None, 'Vat': None, 'Zip': '3901...",11.180298,360 Grad - Bundeswehr Sozialwerk,False
...,...,...,...,...,...,...,...,...,...,...
4995,4995,E9207239053711D4858B00A0244638B0_REDUCED,LTS,False,Farm,46.414400,"{'de': {'Fax': None, 'Vat': None, 'Zip': '3905...",11.233100,Helmut Andergassen,False
4996,4996,E650C0D53DC111D29536004F56000ECA_REDUCED,LTS,False,Farm,46.758332,"{'de': {'Fax': None, 'Vat': None, 'Zip': '3903...",12.042045,Henglerhof,False
4997,4997,ED06EC32FDB15A535FFD5684532BAC16_REDUCED,LTS,False,Farm,46.426005,"{'de': {'Fax': None, 'Vat': None, 'Zip': '3905...",11.252099,Hermann Gatscher,False
4998,4998,BE86941F9712FA664B0DF5E232E6D677_REDUCED,LTS,False,BedBreakfast,46.617405,"{'de': {'Fax': None, 'Vat': None, 'Zip': '3901...",11.134018,Herrenhaus am Grübl,False


In [13]:
accDetail = pd.json_normalize(df3['AccoDetail'])
accDetail = accDetail[[col for col in accDetail if col.startswith('it.')]]

In [14]:
accDetail.columns = accDetail.columns.str.replace('^it.', '')
accDetail.reset_index(inplace=True)
accDetail

/var/folders/y4/87dlj0md68x2q0txn3s2jtmh0000gn/T/ipykernel_58535/3760057915.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  accDetail.columns = accDetail.columns.str.replace('^it.', '')


,index,Fax,Vat,Zip,City,Name,Email,Phone,Mobile,Street,Website,Language,Lastname,Longdesc,Firstname,Shortdesc,CountryCode,NameAddition
0,0,None,None,39100,Bolzano,1100 APARTMENT,None,+39 353 3026699,None,Colle 1/C,None,it,None,None,None,None,IT,None
1,1,None,None,39011,Lana,1477 Reichhalter,None,+39 0473 051050,None,Via Macello 2,http://www.1477reichhalter.com,it,None,None,None,None,IT,None
2,2,None,None,39049,Prati di Vizze,164 apt.,None,,None,Hintere Gasse 164 B,http://www.164apt.com,it,None,None,None,None,IT,None
3,3,None,None,39012,Merano,18,None,+39 339 1055598,None,Via Laurin 32,None,it,None,None,None,None,IT,None
4,4,None,None,39012,Merano,360 Grad - Bundeswehr Sozialwerk,None,+39 0473 232345,None,Via Dante 110,http://www.bundeswehr-sozialwerk.de,it,None,None,None,None,IT,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4995,None,None,39052,Caldaro,Helmut Andergassen,None,+39 0471 964252,None,Pozzo 53,http://www.andergassen-kaltern.it,it,None,None,None,None,IT,None
4996,4996,None,None,39030,Valdaora di Sopra,Henglerhof,None,+39 0474 496016,None,"Via del Borgo, 17",https://www.henglerhof.com,it,None,None,None,None,IT,None
4997,4997,None,None,39052,Caldaro,Hermann Gatscher,None,+39 349 4655718,None,Pianizza di Sotto 10,None,it,None,None,None,None,IT,None
4998,4998,None,None,39011,Lana,Herrenhaus am Grübl,None,+39 339 3607647,None,Via Ultimo 17,None,it,None,None,None,None,IT,None


In [15]:
accomodationFinal = pd.merge(df3, accDetail)
accomodationFinal = accomodationFinal.drop(columns=['AccoDetail', 'index'])
accomodationFinal

,Id,Source,HasRoom,AccoTypeId,Latitude,Longitude,Shortname,IsBookable,Fax,Vat,...,Mobile,Street,Website,Language,Lastname,Longdesc,Firstname,Shortdesc,CountryCode,NameAddition
0,5CEA544EE34639034F07B79D4AEEB603_REDUCED,LTS,False,BedBreakfast,46.477851,11.369939,1100 APARTMENT,False,None,None,...,None,Colle 1/C,None,it,None,None,None,None,IT,None
1,2657B7CBCB85380B253D2FBE28AF100E_REDUCED,LTS,True,BedBreakfast,46.615101,11.143296,1477 Reichhalter,False,None,None,...,None,Via Macello 2,http://www.1477reichhalter.com,it,None,None,None,None,IT,None
2,5E4BEFD440200CE4732A7FA30897C402_REDUCED,LTS,False,BedBreakfast,46.902639,11.461289,164 apt.,False,None,None,...,None,Hintere Gasse 164 B,http://www.164apt.com,it,None,None,None,None,IT,None
3,14AF6F5E67CBEAC34F3223D8DE20D08D_REDUCED,LTS,False,BedBreakfast,46.679733,11.149205,18,False,None,None,...,None,Via Laurin 32,None,it,None,None,None,None,IT,None
4,C93CE627A8497E697C657CF53028F9D2_REDUCED,LTS,False,HotelPension,46.660809,11.180298,360 Grad - Bundeswehr Sozialwerk,False,None,None,...,None,Via Dante 110,http://www.bundeswehr-sozialwerk.de,it,None,None,None,None,IT,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,E9207239053711D4858B00A0244638B0_REDUCED,LTS,False,Farm,46.414400,11.233100,Helmut Andergassen,False,None,None,...,None,Pozzo 53,http://www.andergassen-kaltern.it,it,None,None,None,None,IT,None
4996,E650C0D53DC111D29536004F56000ECA_REDUCED,LTS,False,Farm,46.758332,12.042045,Henglerhof,False,None,None,...,None,"Via del Borgo, 17",https://www.henglerhof.com,it,None,None,None,None,IT,None
4997,ED06EC32FDB15A535FFD5684532BAC16_REDUCED,LTS,False,Farm,46.426005,11.252099,Hermann Gatscher,False,None,None,...,None,Pianizza di Sotto 10,None,it,None,None,None,None,IT,None
4998,BE86941F9712FA664B0DF5E232E6D677_REDUCED,LTS,False,BedBreakfast,46.617405,11.134018,Herrenhaus am Grübl,False,None,None,...,None,Via Ultimo 17,None,it,None,None,None,None,IT,None


In [16]:
accomodationFinal.to_csv("./data/raw_csv/accomodation.csv", index=False)

# Parkplace

In [17]:
df = pd.read_json("./data/parkplace.json")
df2 = pd.DataFrame.from_records(df['data'])
df2.reset_index(inplace=True)
df2

,index,sactive,savailable,scode,scoordinate,smetadata,sname,sorigin,stype
0,0,False,True,102,"{'x': 11.356305, 'y': 46.496449, 'srid': 4326}","{'state': 1, 'capacity': 233, 'mainaddress': '...",P02 - City parking,FAMAS,ParkingStation
1,1,False,True,107,"{'x': 11.351793, 'y': 46.502958, 'srid': 4326}","{'state': 1, 'capacity': 170, 'mainaddress': '...",P07 - Mareccio,FAMAS,ParkingStation
2,2,False,True,112,"{'x': 11.338545, 'y': 46.497757, 'srid': 4326}","{'state': 1, 'capacity': 120, 'mainaddress': '...",P12 - Piazza Tribunale,FAMAS,ParkingStation
3,3,False,True,113,"{'x': 11.338206, 'y': 46.498294, 'srid': 4326}","{'state': 1, 'capacity': 490, 'mainaddress': '...",P13 - Direzional 2,FAMAS,ParkingStation
4,4,False,True,420,"{'x': 11.347308, 'y': 46.494624, 'srid': 4326}","{'state': 1, 'capacity': 70, 'mainaddress': 'V...",@LUNA NOI Technologie Park(test),Legocity,ParkingStation
5,5,False,True,520,"{'x': 11.332848, 'y': 46.479249, 'srid': 4326}","{'state': 1, 'capacity': 50, 'mainaddress': 'V...",@LUNA Parking-Bikesharing(test),Legocity,ParkingStation
6,6,True,True,103,"{'x': 11.355115, 'y': 46.497805, 'srid': 4326}","{'capacity': 403, 'municipality': 'Bolzano - B...",P03 - Piazza Walther,FAMAS,ParkingStation
7,7,True,True,104,"{'x': 11.358216, 'y': 46.500551, 'srid': 4326}","{'capacity': 150, 'municipality': 'Bolzano - B...",P04 - Luna via Molini,FAMAS,ParkingStation
8,8,True,True,105,"{'x': 11.357255, 'y': 46.498174, 'srid': 4326}","{'capacity': 90, 'municipality': 'Bolzano - Bo...",P05 - Laurin,FAMAS,ParkingStation
9,9,True,True,106,"{'x': 11.358907, 'y': 46.497134, 'srid': 4326}","{'capacity': 459, 'municipality': 'Bolzano - B...",P06 - Central Parking via Renon,FAMAS,ParkingStation


In [18]:
coordinate = pd.json_normalize(df2['scoordinate'])
coordinate.drop(columns=['srid'], inplace=True)
coordinate.reset_index(inplace=True)
coordinate.rename(columns={"y": "Latitude", "x": "Longitude"}, inplace=True)
coordinate

,index,Longitude,Latitude
0,0,11.356305,46.496449
1,1,11.351793,46.502958
2,2,11.338545,46.497757
3,3,11.338206,46.498294
4,4,11.347308,46.494624
5,5,11.332848,46.479249
6,6,11.355115,46.497805
7,7,11.358216,46.500551
8,8,11.357255,46.498174
9,9,11.358907,46.497134


In [19]:
parkFinal = pd.merge(df2, coordinate)
parkFinal.drop(columns=['scoordinate', 'smetadata', 'index'], inplace=True)
parkFinal

,sactive,savailable,scode,sname,sorigin,stype,Longitude,Latitude
0,False,True,102,P02 - City parking,FAMAS,ParkingStation,11.356305,46.496449
1,False,True,107,P07 - Mareccio,FAMAS,ParkingStation,11.351793,46.502958
2,False,True,112,P12 - Piazza Tribunale,FAMAS,ParkingStation,11.338545,46.497757
3,False,True,113,P13 - Direzional 2,FAMAS,ParkingStation,11.338206,46.498294
4,False,True,420,@LUNA NOI Technologie Park(test),Legocity,ParkingStation,11.347308,46.494624
5,False,True,520,@LUNA Parking-Bikesharing(test),Legocity,ParkingStation,11.332848,46.479249
6,True,True,103,P03 - Piazza Walther,FAMAS,ParkingStation,11.355115,46.497805
7,True,True,104,P04 - Luna via Molini,FAMAS,ParkingStation,11.358216,46.500551
8,True,True,105,P05 - Laurin,FAMAS,ParkingStation,11.357255,46.498174
9,True,True,106,P06 - Central Parking via Renon,FAMAS,ParkingStation,11.358907,46.497134


In [20]:
parkFinal.to_csv("./data/raw_csv/parkplace.csv", index=False)

# EDA

In [21]:
event = pd.read_csv("./data/raw_csv/event.csv")
event.head(4)

,Id,Source,DateBegin,DateEnd,Latitude,Longitude,Shortname,Title,City,Zip
0,BFEB2DDB0FD54AC9BC040053A5514A92_REDUCED,lts,2022-06-01T00:00:00,2022-06-01T00:00:00,46.644273,11.225259,1000-Stufen-Schlucht im Vinschgau,"Venosta: La ""valle dei 1000 gradini""",Avelengo,39010.0
1,982011E3BE704D22A84F7C2EB40CA8BE_REDUCED,lts,2023-06-02T00:00:00,2023-06-03T00:00:00,46.629600,10.755800,100 Jahre Musikkapelle Kortsch Jubiläumsfest,100 anni banda musicale di corzes,Silandro,39028.0
2,429F65276B0144FCB4ADC354A29C1435_REDUCED,lts,2022-05-28T00:00:00,2022-05-28T00:00:00,46.995900,11.980500,100-Jahr Jubiläum der Musikkapelle Steinhaus,100 anni banda musicale di Cadipietra - festa,Valle Aurina,39030.0
3,10CF80CE69EC42349E4F0B89191F5944_REDUCED,lts,2022-05-29T00:00:00,2022-05-29T00:00:00,46.995940,11.980515,100-Jahr Jubiläum der Musikkapelle Steinhaus,100 anni banda musicale di Cadipietra - festa,Valle Aurina,39030.0


In [22]:
profile = ProfileReport(event, title="Event")
profile.to_file("./EDA/event.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/Users/edoardobianchi/miniforge3/envs/profiling/lib/python3.8/site-packages/multimethod/__init__.py:184: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  return self[tuple(map(self.get_type, args))](*args, **kwargs)


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [23]:
accomodation = pd.read_csv("./data/raw_csv/accomodation.csv")
profile = ProfileReport(accomodation, title="Accomodation")
profile.to_file("./EDA/accomodation.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/Users/edoardobianchi/miniforge3/envs/profiling/lib/python3.8/site-packages/multimethod/__init__.py:184: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  return self[tuple(map(self.get_type, args))](*args, **kwargs)


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [24]:
parkplace = pd.read_csv("./data/raw_csv/parkplace.csv")
profile = ProfileReport(parkplace, title="Parkplace")
profile.to_file("./EDA/parkplace.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/Users/edoardobianchi/miniforge3/envs/profiling/lib/python3.8/site-packages/multimethod/__init__.py:184: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  return self[tuple(map(self.get_type, args))](*args, **kwargs)


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

based on the profiling we can say that ....

we decided to remove these columns because ....
event is fine
parkplace is fine

In [25]:
event

,Id,Source,DateBegin,DateEnd,Latitude,Longitude,Shortname,Title,City,Zip
0,BFEB2DDB0FD54AC9BC040053A5514A92_REDUCED,lts,2022-06-01T00:00:00,2022-06-01T00:00:00,46.644273,11.225259,1000-Stufen-Schlucht im Vinschgau,"Venosta: La ""valle dei 1000 gradini""",Avelengo,39010.0
1,982011E3BE704D22A84F7C2EB40CA8BE_REDUCED,lts,2023-06-02T00:00:00,2023-06-03T00:00:00,46.629600,10.755800,100 Jahre Musikkapelle Kortsch Jubiläumsfest,100 anni banda musicale di corzes,Silandro,39028.0
2,429F65276B0144FCB4ADC354A29C1435_REDUCED,lts,2022-05-28T00:00:00,2022-05-28T00:00:00,46.995900,11.980500,100-Jahr Jubiläum der Musikkapelle Steinhaus,100 anni banda musicale di Cadipietra - festa,Valle Aurina,39030.0
3,10CF80CE69EC42349E4F0B89191F5944_REDUCED,lts,2022-05-29T00:00:00,2022-05-29T00:00:00,46.995940,11.980515,100-Jahr Jubiläum der Musikkapelle Steinhaus,100 anni banda musicale di Cadipietra - festa,Valle Aurina,39030.0
4,9D4618A013144230B8CFF19943335180_REDUCED,lts,2022-09-03T00:00:00,2022-09-03T00:00:00,46.603100,11.520300,10. Barbianer Zwetschkenmeile,10. Miglio delle prugne a Barbiano,Barbiano,39040.0
...,...,...,...,...,...,...,...,...,...,...
95,5C4F708C5B5740D6B2A6DB4A5D6919D3_REDUCED,lts,2016-08-14T00:00:00,2016-08-14T00:00:00,46.724500,12.224300,41. Toblacher Nachtlauf,41° Marcia Notturna,Dobbiaco,39034.0
96,1FC5A2ECA6A14AF2B259F529D46FE41F_REDUCED,lts,2018-02-03T00:00:00,2018-02-04T00:00:00,46.735900,12.222500,41. Volkslanglauf Toblach-Cortina: 50 km Class...,41°Gran Fondo Dobbiaco - Cortina: 50 km Classi...,Dobbiaco,39034.0
97,6D2C08F628F3453C87C25FCB2625A10F_REDUCED,lts,2022-04-30T00:00:00,2022-04-30T00:00:00,46.626700,10.772900,43. Festkonzert der Musikkapelle Kortsch,43° concerto della banda musicale di Corzes,Silandro,39028.0
98,398AAACC7943414E86A866B5ABB8281A_REDUCED,lts,2023-02-12T00:00:00,2023-02-12T00:00:00,46.427100,11.337200,44. Leiferer Fasching,44° Carnevale di Laives,Laives,39055.0


In [27]:
accomodation.drop(columns=["Fax", "Vat", "Email", "Mobile", "Lastname", "Longdesc", "Firstname", "Shortdesc", "NameAddition"], inplace=True)
accomodation.to_csv("./data/cleaned_csv/accomodation_clean.csv", index=False)

In [28]:
accomodation

,Id,Source,HasRoom,AccoTypeId,Latitude,Longitude,Shortname,IsBookable,Zip,City,Name,Phone,Street,Website,Language,CountryCode
0,5CEA544EE34639034F07B79D4AEEB603_REDUCED,LTS,False,BedBreakfast,46.477851,11.369939,1100 APARTMENT,False,39100.0,Bolzano,1100 APARTMENT,+39 353 3026699,Colle 1/C,NaN,it,IT
1,2657B7CBCB85380B253D2FBE28AF100E_REDUCED,LTS,True,BedBreakfast,46.615101,11.143296,1477 Reichhalter,False,39011.0,Lana,1477 Reichhalter,+39 0473 051050,Via Macello 2,http://www.1477reichhalter.com,it,IT
2,5E4BEFD440200CE4732A7FA30897C402_REDUCED,LTS,False,BedBreakfast,46.902639,11.461289,164 apt.,False,39049.0,Prati di Vizze,164 apt.,NaN,Hintere Gasse 164 B,http://www.164apt.com,it,IT
3,14AF6F5E67CBEAC34F3223D8DE20D08D_REDUCED,LTS,False,BedBreakfast,46.679733,11.149205,18,False,39012.0,Merano,18,+39 339 1055598,Via Laurin 32,NaN,it,IT
4,C93CE627A8497E697C657CF53028F9D2_REDUCED,LTS,False,HotelPension,46.660809,11.180298,360 Grad - Bundeswehr Sozialwerk,False,39012.0,Merano,360 Grad - Bundeswehr Sozialwerk,+39 0473 232345,Via Dante 110,http://www.bundeswehr-sozialwerk.de,it,IT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,E9207239053711D4858B00A0244638B0_REDUCED,LTS,False,Farm,46.414400,11.233100,Helmut Andergassen,False,39052.0,Caldaro,Helmut Andergassen,+39 0471 964252,Pozzo 53,http://www.andergassen-kaltern.it,it,IT
4996,E650C0D53DC111D29536004F56000ECA_REDUCED,LTS,False,Farm,46.758332,12.042045,Henglerhof,False,39030.0,Valdaora di Sopra,Henglerhof,+39 0474 496016,"Via del Borgo, 17",https://www.henglerhof.com,it,IT
4997,ED06EC32FDB15A535FFD5684532BAC16_REDUCED,LTS,False,Farm,46.426005,11.252099,Hermann Gatscher,False,39052.0,Caldaro,Hermann Gatscher,+39 349 4655718,Pianizza di Sotto 10,NaN,it,IT
4998,BE86941F9712FA664B0DF5E232E6D677_REDUCED,LTS,False,BedBreakfast,46.617405,11.134018,Herrenhaus am Grübl,False,39011.0,Lana,Herrenhaus am Grübl,+39 339 3607647,Via Ultimo 17,NaN,it,IT


In [29]:
parkplace

,sactive,savailable,scode,sname,sorigin,stype,Longitude,Latitude
0,False,True,102,P02 - City parking,FAMAS,ParkingStation,11.356305,46.496449
1,False,True,107,P07 - Mareccio,FAMAS,ParkingStation,11.351793,46.502958
2,False,True,112,P12 - Piazza Tribunale,FAMAS,ParkingStation,11.338545,46.497757
3,False,True,113,P13 - Direzional 2,FAMAS,ParkingStation,11.338206,46.498294
4,False,True,420,@LUNA NOI Technologie Park(test),Legocity,ParkingStation,11.347308,46.494624
5,False,True,520,@LUNA Parking-Bikesharing(test),Legocity,ParkingStation,11.332848,46.479249
6,True,True,103,P03 - Piazza Walther,FAMAS,ParkingStation,11.355115,46.497805
7,True,True,104,P04 - Luna via Molini,FAMAS,ParkingStation,11.358216,46.500551
8,True,True,105,P05 - Laurin,FAMAS,ParkingStation,11.357255,46.498174
9,True,True,106,P06 - Central Parking via Renon,FAMAS,ParkingStation,11.358907,46.497134


# Dependency Discover